## pytorch_seresnext101-32x4d+kfold+focal loss v3.0


* fold = 4
* epoch = 3
* γ=0.4

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import time
import numpy as np
import math
import pandas as pd
from PIL import Image, ImageOps, ImageFilter
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import random
import datetime
import os

import cv2  
from sklearn import preprocessing 
from sklearn.model_selection import KFold
import h5py

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose,RandomContrast,RandomBrightness,Resize
)
import albumentations

In [3]:
from torch.utils.data import DataLoader
from prefetch_generator import BackgroundGenerator

class DataLoaderX(DataLoader):

    def __iter__(self):
        return BackgroundGenerator(super().__iter__())

In [4]:
# os.chdir("../input/pretrained_PyTorch/")
path = os.getcwd()
print(path)

/home/user1/Downloads/iMet-Collection-2019-FGVC6


In [5]:
train = pd.read_csv(path+"/train.csv")
lable = pd.read_csv(path+"/labels.csv")
test = pd.read_csv(path+"/sample_submission.csv")

In [6]:
lable_length = len(lable)
train_length = len(train)
test_length = len(test)
print(train_length)
print(lable_length)
print(test_length)

109237
1103
38801


In [7]:
print(np.array(lable)[397])
print(np.array(lable)[398])
c_length = len(np.array(lable)[:398])
t_length = len(np.array(lable)[398:])
print(c_length)
print(t_length)

[397 'culture::zurich']
[398 'tag::abbies']
398
705


In [8]:
#np.array(test)

In [9]:
# test.head

In [10]:
#np.array(train)

In [11]:
def augment(p=.5):
    return Compose([
        HorizontalFlip(p=0.5),
        OneOf([
            RandomBrightness(0.1, p=1),
            RandomContrast(0.1, p=1),
        ], p=0.3),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.0, rotate_limit=15, p=0.3),
        IAAAdditiveGaussianNoise(p=0.3)
        #Resize(220,220)
    ], p=p)
def strong_aug(p=.5):
    return Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p)

aug = augment(0.2)

In [12]:
def creatData(train,lable_length):
    train = np.array(train)
    trainA_data = []
    lab_data = []
    #trainC_data = []
    #trainT_data = []
    for t in range(train_length):
        v = np.zeros(lable_length)
        #print(train[t,1])
        lab = []
        for s in train[t,1].split(" "):
            #print(s)
            v[int(s)] = 1
            lab.append(int(s))
            
#         img = Image.open(path+"/train/"+format(train[t,0])+'.png')  # PIL image
#         img = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)
#         img = aug(image = img)['image']
#         #openCV to PIL
#         img = Image.fromarray(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
        
        trainA_data.append([train[t,0],v])
        lab_data.append([train[t,0],np.array(lab)])

        #trainC_data.append([train[t,0],v[:c_length]])
        #trainT_data.append([train[t,0],v[c_length:]])
    return np.array(trainA_data),np.array(lab_data)
    #return np.array(trainA_data)#,np.array(trainC_data),np.array(trainT_data)

In [13]:
train_a,train_lab = creatData(train,lable_length)
#train_a,train_c,train_t = creatData(train,lable_length)
#train_t.shape()

In [14]:
train_lab[0]

array(['1000483014d91860', array([147, 616, 813])], dtype=object)

### Remove the item that only have one feature (not work well)

In [15]:
def dfilter(data,th):
    dfilter = []
    for i in range(len(data)):
        if train_a[i][1].sum()>th:
            dfilter.append(train_a[i])
    return np.array(dfilter)

In [16]:
datafilter = dfilter(train_a,1) # remain feature great then 1

In [17]:
print("amount of image before: {}".format(len(train_a)))

print("amount of image after: {}".format(len(datafilter)))

amount of image before: 109237
amount of image after: 104913


In [18]:
# not use
datafilter = train_a

In [19]:
datafilter[0]

array(['1000483014d91860', array([0., 0., 0., ..., 0., 0., 0.])],
      dtype=object)

In [20]:
image_resize = 220
data_transforms2 = transforms.Compose([
    transforms.Resize((image_resize,image_resize)),
    #transforms.RandomResizedCrop(250),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
            [0.485, 0.456, 0.406], 
            [0.229, 0.224, 0.225])
    ])


data_transforms = transforms.Compose([
    transforms.Resize((image_resize,image_resize)),
    #transforms.RandomResizedCrop(250),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize(
    #        [0.485, 0.456, 0.406], 
    #        [0.229, 0.224, 0.225])
    ])

train_transformer = transforms.Compose([
    transforms.Resize((128,128)),              # resize the image to 
    #transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.ToTensor(),           # transform it into a PyTorch Tensor
    #transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
])

In [21]:
# top_transforms = transforms.Compose([
#     augment()['image'],
#     transforms.Resize((image_resize,image_resize)),
#     HorizontalFlip(p=0.5),
#     OneOf([
#         RandomBrightness(0.1, p=1),
#         RandomContrast(0.1, p=1),
#     ], p=0.3),
#     ShiftScaleRotate(shift_limit=0.1, scale_limit=0.0, rotate_limit=15, p=0.3),
#     IAAAdditiveGaussianNoise(p=0.3),
#     #transforms.RandomResizedCrop(250),
#     #transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(
#             [0.485, 0.456, 0.406], 
#             [0.229, 0.224, 0.225])
#     ])

In [22]:
def load2mem(in_file,transform):
    print("Load data from .h5 file...")
    t = time.time()
    prefile = h5py.File(in_file, 'r')
    preloadimg = []
    preloadleb = prefile['train_labels'][:,:][:20000]
    #print(prefile['train_img'][0,:,:])
#     for i in range(len(preloadleb)):
#         img = Image.fromarray(prefile['train_img'][i,:,:])
#         preloadimg.append(img)
    preloadimg = prefile['train_img'][:,:,:][:20000]
    
    print("Done")
    print("Take",datetime.timedelta(seconds = int(time.time()-t)),"seconds")
    return preloadimg, preloadleb


In [23]:
#a,b = load2mem("all_data_109237v1.h5", data_transforms2)

In [24]:
class dataset_h5(torch.utils.data.Dataset):
    def __init__(self, in_file, transform, kdata):
        super(dataset_h5, self).__init__()

        self.file = h5py.File(in_file, 'r')
        self.n_images = len(kdata)
        self.kdata = kdata
        self.transform = transform
        print("Load data from .h5 file...")
        t = time.time()
        self.preloadimg = self.file['train_img'][self.kdata,:,:]
        self.preloadleb = self.file['train_labels'][self.kdata,:]
#         self.preloadimg = [a[i] for i in self.kdata]
#         self.preloadleb = b[self.kdata]
        #print("Take",datetime.timedelta(seconds = int(time.time()-t)),"seconds")

    def __getitem__(self, index):
        
        input_img = Image.fromarray(self.preloadimg[index])
        
#         imgv = cv2.cvtColor(np.asarray(input_img),cv2.COLOR_RGB2BGR)
#         imgv = aug(image = imgv)['image']
#         #openCV to PIL
#         input_img = Image.fromarray(cv2.cvtColor(imgv,cv2.COLOR_BGR2RGB))
        
        input_img = self.transform(input_img)
        
        input_lab = self.preloadleb[index]
        return input_img,input_lab

    def __len__(self):
        return self.n_images

In [25]:
class trainDataset(Dataset):
    def __init__(self, train_lib, transform,transform2):
        self.filenames = train_lib[:,0]
        self.labels = train_lib[:,1]
        self.transform = transform
        self.transform2 = transform2
        #self.new_feature = 

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img = Image.open(path+"/train/"+format(self.filenames[idx])+'.png')  # PIL image
        
        #PIL to openCV
        imgv = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)
        imgv = aug(image = imgv)['image']
        #openCV to PIL
        img = Image.fromarray(cv2.cvtColor(imgv,cv2.COLOR_BGR2RGB))
        
        #image= image.filter(ImageFilter.EDGE_ENHANCE)
        #image2 = image.filter(ImageFilter.FIND_EDGES)
        image = self.transform(img)
        image2 = self.transform2(img)
        #return image, self.labels[idx]
        return image,image2, self.labels[idx]


In [26]:
class testDataset(Dataset):
    def __init__(self, test_lib, transform,transform2):
        test_lib = np.array(test_lib)
        self.filenames = test_lib[:,0]
        #self.labels = test_lib[:,1]
        self.transform = transform
        self.transform2 = transform2

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img = Image.open(path+"/test/"+format(self.filenames[idx])+'.png')  # PIL image
        #image= image.filter(ImageFilter.EDGE_ENHANCE)
        #image2 = image.filter(ImageFilter.FIND_EDGES)
        image = self.transform(img)
        image2 = self.transform2(img)
        #return image,self.filenames[idx]
        return image,image2,self.filenames[idx]

In [27]:
#trainA_dataloader = DataLoader(dataset_h5("all_data_109237v1.h5", data_transforms2),batch_size=128, shuffle=True,num_workers=2, pin_memory=True)
#trainC_dataloader = DataLoader(trainDataset(train_c, data_transforms,data_transforms2),batch_size=32, shuffle=True,num_workers=2, pin_memory=True)
#trainT_dataloader = DataLoader(trainDataset(train_t, data_transforms,data_transforms2),batch_size=32, shuffle=True,num_workers=2, pin_memory=True)

In [40]:
score_dataloader = DataLoader(dataset_h5("all_data_109237v1.h5", data_transforms2, [i for i in range(1000)]),batch_size=80, shuffle=False,num_workers=16, pin_memory=True)

Load data from .h5 file...


In [41]:
test_dataloader = DataLoaderX(testDataset(test, data_transforms,data_transforms2),batch_size=128,shuffle=False,num_workers=2, pin_memory=True)

IndexError: too many indices for array

In [30]:
# plt.figure(figsize=(10,10))
# plt.subplot(2,2,1)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[15][0]).convert('RGB'))
# plt.subplot(2,2,2)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[15][1]).convert('RGB'))
# plt.subplot(2,2,3)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[29][0]).convert('RGB'))
# plt.subplot(2,2,4)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[29][1]).convert('RGB'))


In [31]:
#(np.array(trainC_dataloader.dataset[4][0]).shape)

In [32]:
import pretrainedmodels

model = pretrainedmodels.__dict__["se_resnext101_32x4d"](num_classes=1000, pretrained='imagenet')
model.last_linear = nn.Linear(in_features=2048, out_features=lable_length, bias=True)

if torch.cuda.device_count() > 1:
    # device_ids has a default : all
    model = torch.nn.DataParallel(model, device_ids=[0,1]) 

model.to(device)

DataParallel(
  (module): SENet(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [33]:
from torchsummary import summary
summary(model, (3, 220, 220))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 110, 110]           9,408
       BatchNorm2d-2         [-1, 64, 110, 110]             128
              ReLU-3         [-1, 64, 110, 110]               0
         MaxPool2d-4           [-1, 64, 55, 55]               0
            Conv2d-5         [-1, 64, 110, 110]           9,408
            Conv2d-6          [-1, 128, 55, 55]           8,192
       BatchNorm2d-7          [-1, 128, 55, 55]             256
              ReLU-8          [-1, 128, 55, 55]               0
       BatchNorm2d-9         [-1, 64, 110, 110]             128
             ReLU-10         [-1, 64, 110, 110]               0
        MaxPool2d-11           [-1, 64, 55, 55]               0
           Conv2d-12          [-1, 128, 55, 55]           8,192
      BatchNorm2d-13          [-1, 128, 55, 55]             256
             ReLU-14          [-1, 128,

In [34]:
model = torch.load('net_se_resnext101_t1.pkl')

In [35]:
def findPre(output,gate):
    a = ''
    output = np.array(output)
    m = np.max(output)

    for i in range(len(output)):
        #s = np.where(v[i] > 0.95, 1, 0)
        if output[i]>gate:
            #print(output[i])
            a = a + format(i)+' '
            
    #print(a)
    return a
    

In [44]:
def test(model,dloader,threshold):
    model.eval().cuda()
    #lengthD = len(dloader.dataset)
    ans = []
    out = []
    for step, (x, y) in enumerate(dloader):
        data = Variable(x).cuda()
        #data2 = Variable(x2).cuda()
        #data = Variable(x)
        target = y
        #output = model(data,data2)
        with torch.no_grad():
            output = model(data).detach()
        
        #data.cpu()   # batch x
        #data2.cpu()   # batch x
        #target.cpu()   # batch y
        #torch.cuda.empty_cache()
        
        v = output.cpu()
        v = torch.sigmoid(v)
        
        v = torch.sigmoid(v)
        v = np.array(v)
        v = preprocessing.minmax_scale(v, feature_range=(0,1),axis=1)
        for i in range(len(v)):
            out.append(np.where(v[i] > threshold, 1, 0))
            s = findPre(v[i],threshold)
            ans.append([target[i],s])
        if step %10 == 0:
            print('[{}/{} ({:.1f}%)]'.format(step * len(data), 
                                        len(dloader.dataset),
                                        100.*step/len(dloader)))
            
        data.cpu()
        #data2.detach()
        #target.detach()
        torch.cuda.empty_cache()
    print("Finish")
    return ans,out
    

In [45]:
from sklearn.metrics import fbeta_score

def makeScore(pre,ans):
    pre = np.array(pre)
    va = fbeta_score(y_pred=pre, y_true=ans, beta=2, average="samples")
    print("Score : {:.5f}".format(va))
    return va
    

In [46]:
def findThreshold():
    score = []
    candidates = np.arange(0, 1.0, 0.01)
    for th in candidates:
        print("Threshold : {:.2f}".format(th))
        _,pre = test(model = model,dloader = score_dataloader,threshold = th)
        #return pre
        score.append(makeScore(np.array(pre),np.array(train_a[:1000,1].tolist())))
        print("=============================")
    pm = np.array(score).argmax()
    best_th, best_score = candidates[pm], score[pm]
    return best_th, best_score

In [47]:
bt, bs = findThreshold()
print("Best Threshold : {:.2f}".format(bt))
print("Best Score : {:.5f}".format(bs))

Threshold : 0.00
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.01474
Threshold : 0.01
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.25405
Threshold : 0.02
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.35171
Threshold : 0.03
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.41979
Threshold : 0.04
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.47201
Threshold : 0.05
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.51354
Threshold : 0.06
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.54441
Threshold : 0.07
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.57278
Threshold : 0.08
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.59629
Threshold : 0.09
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.61567
Threshold : 0.10
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.63423
Threshold : 0.11
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.65029
Threshold : 0.12
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.66201
Threshold : 0.13
[0/1000 

[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.55743
Threshold : 0.79
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.55356
Threshold : 0.80
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.54952
Threshold : 0.81
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.54384
Threshold : 0.82
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.53867
Threshold : 0.83
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.53080
Threshold : 0.84
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.52780
Threshold : 0.85
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.52181
Threshold : 0.86
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.51508
Threshold : 0.87
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.50839
Threshold : 0.88
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.50027
Threshold : 0.89
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.49183
Threshold : 0.90
[0/1000 (0.0%)]
[800/1000 (76.9%)]
Finish
Score : 0.48566
Threshold : 0.91
[0/1000 (0.0%)]
[800/1000

In [ ]:
torch.cuda.empty_cache()
!nvidia-smi

In [ ]:
sub,_ = test(model = model,dloader = test_dataloader,threshold = bt)

In [ ]:
#print(sub)

In [ ]:
sub =  pd.DataFrame(sub)

In [ ]:
sub = sub.rename(index=str, columns={0: "id", 1: "attribute_ids"})

In [ ]:
sub.head

In [ ]:
sub.to_csv('submission_t1.csv', index=False)